In [6]:
import os 
dirs = [ os.path.join(os.path.abspath("./") , x) for x in os.listdir() if os.path.isdir(x) and "Zn" in x]
ex =  dirs[0]
infile = [ os.path.join(ex , x) for x in os.listdir(ex) if "Zn" in x ][0]
infile

'c:\\Users\\PSID_PC_20\\Desktop\\[00]Projects\\PSID_server_room\\qe\\zinc_surface_water.relax\\Zn_002_water\\Zn_002_water.relax.in'

In [7]:
import os
import numpy as np
from scipy.spatial import KDTree

# 설정: 원자 파일 이름과 출력 파일
input_file = infile
output_file = os.path.join(ex, "atomic_positions_hop_based.in")

# 흡착 중심 좌표 (흡착체가 있는 위치)
adsorption_center = np.array([3.098737, 4.415163, 5.5375210])

# 홉별 Relax 설정 (최대 3 홉까지 Relax)
max_relax_hop = 2  # 1~2 홉까지 Relax, 3 홉 이상 고정

# 원자 데이터 저장 리스트
elements = []
coordinates = []

# 파일 읽기
with open(input_file, "r") as f:
    lines = f.readlines()

for line in lines:
    split_line = line.strip().split()
    if len(split_line) == 4:  # 원자 정보가 있는 줄만 처리
        element, x, y, z = split_line
        coordinates.append([float(x), float(y), float(z)])
        elements.append(element)

# NumPy 배열로 변환
coordinates = np.array(coordinates)

# KDTree를 사용하여 홉(shell) 계산
tree = KDTree(coordinates)

# 흡착 중심과 가장 가까운 원자 찾기 (흡착 사이트 설정)
_, nearest_index = tree.query(adsorption_center)
first_shell_indices = tree.query_ball_point(coordinates[nearest_index], r=2.5)  # 1 홉 범위

# 2 홉 찾기: 1 홉 원자들에서 추가적으로 거리 계산
second_shell_indices = set()
for idx in first_shell_indices:
    neighbors = tree.query_ball_point(coordinates[idx], r=2.5)  # 2 홉 범위
    second_shell_indices.update(neighbors)

# 3 홉 이상 원자들
third_shell_indices = set(range(len(coordinates))) - set(first_shell_indices) - set(second_shell_indices)

# 결과 저장
output_lines = []
for i, (element, coord) in enumerate(zip(elements, coordinates)):
    x, y, z = coord

    if i in first_shell_indices:  # 1 홉 (Relax)
        output_lines.append(f"{element}  {x:.6f}  {y:.6f}  {z:.6f}  1  1  1\n")
    elif i in second_shell_indices and max_relax_hop >= 2:  # 2 홉 (조건부 Relax)
        output_lines.append(f"{element}  {x:.6f}  {y:.6f}  {z:.6f}  1  1  1\n")
    else:  # 3 홉 이상 (고정)
        output_lines.append(f"{element}  {x:.6f}  {y:.6f}  {z:.6f}  0  0  0\n")

# 새로운 파일로 저장
with open(output_file, "w") as f:
    f.writelines(output_lines)

print(f"✅ 완료! 새로운 홉 기반 Relax 설정 파일 {output_file}이(가) 생성되었습니다.")


✅ 완료! 새로운 홉 기반 Relax 설정 파일 c:\Users\PSID_PC_20\Desktop\[00]Projects\PSID_server_room\qe\zinc_surface_water.relax\Zn_002_water\atomic_positions_hop_based.in이(가) 생성되었습니다.


## 고정된 분자 시각화 matplotlib

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import json

# 원소 색상 파일 불러오기
with open("element_colors.json", "r") as f:
    element_colors = json.load(f)

# ATOMIC_POSITIONS 파일 입력
input_file = "atomic_positions.in"

# 원자 데이터 저장 리스트
elements = []
coordinates = []
relax_flags = []

# 파일 읽기
with open(input_file, "r") as f:
    lines = f.readlines()

for line in lines:
    split_line = line.strip().split()
    if len(split_line) == 7:  # 원자 정보가 있는 줄만 처리
        element, x, y, z, rx, ry, rz = split_line
        coordinates.append([float(x), float(y), float(z)])
        elements.append(element)
        relax_flags.append((int(rx), int(ry), int(rz)))

# NumPy 배열 변환
coordinates = np.array(coordinates)
relax_flags = np.array(relax_flags)

# Relax 여부 판단 (1 1 1 인 경우 색깔 변경)
is_relaxed = np.all(relax_flags == (1, 1, 1), axis=1)

# 플롯 생성
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

# Relaxed 원자 (원소별 색상 적용)
for element in set(elements):  # 중복 없이 원소 리스트 추출
    element_mask = (np.array(elements) == element) & is_relaxed  # 해당 원소 & Relax된 원자만 선택
    if np.any(element_mask):  # 존재하는 경우만 추가
        ax.scatter(coordinates[element_mask, 0], coordinates[element_mask, 1], coordinates[element_mask, 2],
                   color=element_colors.get(element, "black"), label=f"{element} (Relaxed)", s=50, edgecolors='k')

# Fixed 원자 (회색)
ax.scatter(coordinates[~is_relaxed, 0], coordinates[~is_relaxed, 1], coordinates[~is_relaxed, 2], 
           color='gray', label="Fixed Atoms", s=30, alpha=0.5)

# 라벨 및 스타일 설정
ax.set_xlabel("X (Å)")
ax.set_ylabel("Y (Å)")
ax.set_zlabel("Z (Å)")
ax.legend()
plt.title("Relaxed (Colored by Element) vs Fixed (Gray) Atoms")

# 그래프 출력
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'atomic_positions.in'

In [9]:
import numpy as np
import plotly.graph_objects as go
import json

# ATOMIC_POSITIONS 파일 입력
input_file = output_file

# 원소 색상 파일 불러오기
with open("element_colors.json", "r") as f:
    element_colors = json.load(f)

# 원자 데이터 저장 리스트
elements = []
coordinates = []
relax_flags = []

# 파일 읽기
with open(input_file, "r") as f:
    lines = f.readlines()

for line in lines:
    split_line = line.strip().split()
    
    # 7개 값이 있어야 정상적인 원자 데이터로 간주
    if len(split_line) == 7:
        element, x, y, z, rx, ry, rz = split_line
        coordinates.append([float(x), float(y), float(z)])
        elements.append(element)
        relax_flags.append([int(rx), int(ry), int(rz)])

# numpy 배열 변환
coordinates = np.array(coordinates)
relax_flags = np.array(relax_flags)

# Relax 여부 판단 (모든 좌표에서 1 1 1인 경우 Relax로 분류)
is_relaxed = np.all(relax_flags == [1, 1, 1], axis=1)

# 3D 시각화
fig = go.Figure()

# Relaxed 원자 (원소별 색상)
for element in set(elements):  # 중복 없이 원소 리스트 추출
    element_mask = (np.array(elements) == element) & is_relaxed  # 해당 원소 & Relax된 원자만 선택
    if np.any(element_mask):  # 존재하는 경우만 추가
        fig.add_trace(go.Scatter3d(
            x=coordinates[element_mask, 0],
            y=coordinates[element_mask, 1],
            z=coordinates[element_mask, 2],
            mode='markers',
            marker=dict(size=6, color=element_colors.get(element, "black"), opacity=1.0),
            name=f"{element} (Relaxed)"
        ))

# Fixed 원자 (회색)
fig.add_trace(go.Scatter3d(
    x=coordinates[~is_relaxed, 0], 
    y=coordinates[~is_relaxed, 1], 
    z=coordinates[~is_relaxed, 2],
    mode='markers', marker=dict(size=5, color='gray', opacity=0.5),
    name="Fixed Atoms"
))

# 3D 레이아웃 설정
fig.update_layout(
    title="Interactive 3D Atom Visualization (Color by Element)",
    scene=dict(
        xaxis_title="X (Å)",
        yaxis_title="Y (Å)",
        zaxis_title="Z (Å)",
    ),
    margin=dict(l=0, r=0, b=0, t=40),
    height=700
)

# 인터랙티브 플롯 출력
fig.show()


In [9]:
relax_flags

array([], dtype=float64)

In [23]:
import plotly
print(plotly.__version__)


5.24.1
